# V27: TRICERATOPS(+ ) higher-fidelity run (standard preset)

Runs TRICERATOPS(+ ) with a higher-fidelity configuration than the fast preset:
- uses the **gated sectors (82+83)**
- uses the **stable detrend baseline** from V35
- includes the **PHARO Kcont** AO contrast curve
- increases Monte Carlo draws to reduce variance

This is intended to produce a more stable reportable FPP than the fast-grid runs.


In [ ]:
from __future__ import annotations

import contextlib
import json
import sys
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np

tutorial_dir = Path('docs/tutorials/tutorial_toi-5807-incremental').resolve()
sys.path.insert(0, str(tutorial_dir))

import toi5807_shared as sh

btv = sh.btv

step_id = '41_v27_triceratops_standard_with_contrast_curve'
run_out_dir, docs_out_dir = sh.artifact_dirs(step_id=step_id)

sectors = [82, 83]

# Baseline detrend config (V35)
bin_hours = 6.0
duration_buffer_factor = 2.0
sigma_clip = 5.0

contrast_curve_path = Path('docs/tutorials/data/tic188646744/PHARO_Kcont_plot.tbl')
contrast_curve = btv.load_contrast_curve_exofop_tbl(contrast_curve_path, filter='Kcont')

# Step-specific cache storing detrended LCs under flux_type='pdcsap'
cache_dir = run_out_dir / 'cache_pdcsap_detrended'

# Seed TRICERATOPS Target + TRILEGAL cache from V22 to avoid brittle upstream form changes.
seed_target_cache_src = Path(
    'persistent_cache/tutorial_toi-5807-incremental/36_v22_triceratops_fpp/'
    'cache_pdcsap_detrended/triceratops/target_cache/tic_188646744__sectors_82-83.pkl'
)
seed_trilegal_csv_src = Path(
    'persistent_cache/tutorial_toi-5807-incremental/36_v22_triceratops_fpp/'
    'cache_pdcsap_detrended/triceratops/188646744_TRILEGAL.csv'
)
tc_dir = cache_dir / 'triceratops' / 'target_cache'
tc_dir.mkdir(parents=True, exist_ok=True)
if seed_target_cache_src.exists():
    (tc_dir / seed_target_cache_src.name).write_bytes(seed_target_cache_src.read_bytes())
tr_dir = cache_dir / 'triceratops'
tr_dir.mkdir(parents=True, exist_ok=True)
if seed_trilegal_csv_src.exists():
    (tr_dir / seed_trilegal_csv_src.name).write_bytes(seed_trilegal_csv_src.read_bytes())

# Build detrended per-sector dataset
base = sh.load_dataset()
detrended_by_sector = {
    int(s): sh.detrend_transit_masked_bin_median(
        base.lc_by_sector[int(s)],
        bin_hours=bin_hours,
        duration_buffer_factor=duration_buffer_factor,
        sigma_clip=sigma_clip,
    )
    for s in sectors
}

from bittr_tess_vetter.api.datasets import LocalDataset

ds_det = LocalDataset(
    schema_version=base.schema_version,
    root=base.root,
    lc_by_sector=detrended_by_sector,
    tpf_by_sector={},
    artifacts=base.artifacts,
)

cache = btv.hydrate_cache_from_dataset(
    dataset=ds_det,
    tic_id=sh.TIC_ID,
    flux_type='pdcsap',
    cache_dir=cache_dir,
    sectors=sectors,
)

stitched_det = sh.stitch_pdcsap_sectors(ds_det, sectors)
depth_ppm, depth_err_ppm = sh.estimate_depth_ppm(stitched_det)

# Higher-fidelity overrides (more draws, but keep an empirical noise floor for stability)
overrides = {
    'mc_draws': 200000,
    'max_points': 3000,
    'window_duration_mult': 2.0,
    'min_flux_err': 5e-5,
    'use_empirical_noise_floor': True,
}

stdout_path = run_out_dir / 'triceratops_stdout.log'
stderr_path = run_out_dir / 'triceratops_stderr.log'

with (
    stdout_path.open('w', encoding='utf-8') as out,
    stderr_path.open('w', encoding='utf-8') as err,
    contextlib.redirect_stdout(out),
    contextlib.redirect_stderr(err),
):
    result = btv.calculate_fpp(
        cache=cache,
        tic_id=sh.TIC_ID,
        period=sh.PERIOD_DAYS,
        t0=sh.T0_BTJD,
        depth_ppm=float(depth_ppm),
        duration_hours=sh.DURATION_HOURS,
        sectors=sectors,
        stellar_radius=sh.STELLAR.radius,
        stellar_mass=sh.STELLAR.mass,
        preset='standard',
        overrides=overrides,
        timeout_seconds=900,
        replicates=1,
        seed=123,
        contrast_curve=contrast_curve,
    )

payload = {
    'cache_dir': str(cache_dir),
    'cached_flux_type': 'pdcsap',
    'sectors': sectors,
    'detrend': {
        'bin_hours': bin_hours,
        'duration_buffer_factor': duration_buffer_factor,
        'sigma_clip': sigma_clip,
    },
    'depth_ppm_used': float(depth_ppm),
    'depth_err_ppm_est': float(depth_err_ppm),
    'triceratops_overrides': overrides,
    'contrast_curve_path': str(contrast_curve_path),
    'fpp_result': result,
    'logs': {'stdout': str(stdout_path), 'stderr': str(stderr_path)},
}

json_name = 'triceratops_standard_with_contrast_curve.json'
(run_out_dir / json_name).write_text(json.dumps(payload, indent=2, sort_keys=True))
if docs_out_dir is not None:
    (docs_out_dir / json_name).write_text(json.dumps(payload, indent=2, sort_keys=True))

summary = {
    'depth_ppm_used': round(float(depth_ppm), 3),
    'fpp': result.get('fpp'),
    'nfpp': result.get('nfpp'),
    'disposition': result.get('disposition'),
    'replicates': result.get('replicates'),
}
print(json.dumps(summary, indent=2, sort_keys=True))

# quick scenario-prob plot
probs = {
    'planet': result.get('prob_planet'),
    'EB': result.get('prob_eb'),
    'BEB': result.get('prob_beb'),
    'NEB': result.get('prob_neb'),
    'NTP': result.get('prob_ntp'),
}
labels = list(probs.keys())
vals = [float(probs[k]) if probs[k] is not None else np.nan for k in labels]
fig, ax = plt.subplots(figsize=(7.5, 3.8), dpi=150)
ax.bar(labels, vals, color=['#2ca02c'] + ['#d62728'] * 4)
ax.set_ylim(0, 1)
ax.set_ylabel('Probability')
ax.set_title('V27: TRICERATOPS(+ ) scenario probabilities (higher-fidelity)')
for i, v in enumerate(vals):
    if np.isfinite(v):
        ax.text(i, min(0.98, v + 0.03), f"{v:.3f}", ha='center', va='bottom', fontsize=9)
fpp = result.get('fpp')
nfpp = result.get('nfpp')
ax.text(
    0.98,
    0.98,
    f"FPP={fpp:.3g}\nNFPP={nfpp:.3g}" if fpp is not None and nfpp is not None else 'FPP/NFPP unavailable',
    ha='right',
    va='top',
    transform=ax.transAxes,
    bbox={'boxstyle': 'round,pad=0.25', 'fc': 'white', 'ec': '0.7'},
)
ax.text(
    0.02,
    0.98,
    'AO constraint: PHARO Kcont',
    ha='left',
    va='top',
    transform=ax.transAxes,
    bbox={'boxstyle': 'round,pad=0.25', 'fc': 'white', 'ec': '0.7'},
)
fig.tight_layout()
png_name = 'V27_triceratops_standard.png'
run_png = run_out_dir / png_name
fig.savefig(run_png)
if docs_out_dir is not None:
    fig.savefig(docs_out_dir / png_name)
plt.close(fig)
print(json.dumps({'run_plot_path': str(run_png), 'docs_plot_path': str((docs_out_dir / png_name) if docs_out_dir is not None else None)}, indent=2, sort_keys=True))


<details>
<summary><b>Expected Output</b></summary>

This step writes:
- JSON: `docs/tutorials/artifacts/tutorial_toi-5807-incremental/41_v27_triceratops_standard_with_contrast_curve/triceratops_standard_with_contrast_curve.json`
- Plot: `docs/tutorials/artifacts/tutorial_toi-5807-incremental/41_v27_triceratops_standard_with_contrast_curve/V27_triceratops_standard.png`

Exact numeric results can vary slightly with upstream catalogs / priors.

```text
{
  "depth_ppm_used": 210.635,
  "fpp": 3e-06,
  "nfpp": 0.0,
  "disposition": "VALIDATED",
  "prob_planet": 0.987428
}
```

</details>


## Plot

<img src="../artifacts/tutorial_toi-5807-incremental/41_v27_triceratops_standard_with_contrast_curve/V27_triceratops_standard.png" width="820" />


<details>
<summary><b>Analysis</b></summary>

- **What changed vs fast runs:** more Monte Carlo draws (and no aggressive downsampling/windowing) reduces variance and makes the returned FPP more stable.
- **How to interpret:** compare the resulting FPP/NFPP to V22/V23 and ensure they are consistent in direction. Large disagreements are a warning to audit depth/preprocessing and cached target assumptions.
- **Caveat:** this remains conditional on the adopted depth (preprocessing baseline). If depth changes, re-run.

</details>
